In [1]:
import os
import sys
import numpy as np
from PIL import Image
from torch.utils import data
import torch
from torchvision import transforms
import math

sys.path.append('/home/ash/deeplab/')


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import cv2

In [3]:
def predict_third_point(x1,x2,theta):
    '''
    预测第三个点
    '''
    # np.cos输入是弧度制
    return x1*x2 / (2*x1*np.cos(theta) - x2)


def conpare_third_point(x, x_predict, threshod=0.4):
    # 区分左右跳变
    if (x-x_predict)>threshod:
        return 1
    elif (x_predict - x)>threshod:
        return -1
    else:
        return 0


# 2度以上认为是大障碍物
FOV_THRE = 2
FOV_THRE = np.deg2rad(FOV_THRE)
LIDAR_FOV_RESOLUSION = math.radians(0.2)


def find_obstacle_by_breakpoint(breakpoints, obstacle, FOV_threshold = FOV_THRE):

    FOV_threshold_pixel = round(FOV_threshold / LIDAR_FOV_RESOLUSION)
    """
    args：

        breakpoints 输入的一条id的lidar
        obstacle 输出的障碍物属性标签
        FOV_threshold_pixel 阈值，大于 2°/0.2°认为是大障碍物

    TODO 注意 ：

        角度遍历是逆时针的

        这里假设如下，有问题再翻转一下
            从-1到1，负障碍物
            从1到-1，正障碍物

    应该防止的BUG：

        连续出现三个及以上的+-1跳变，说明障碍物表面复杂，或者有噪声
    """
    # 跳变点状态
    positiveFlag = 0
    negativeFlag = 0
    # 当前状态索引
    positiveIndex = 0
    negativeIndex = 0
    # 计数器
    cnt = 0

    for i in breakpoints:
        cnt = cnt + 1
        if i == -1:
            negativeFlag = 1
            negativeIndex = cnt
        elif i == 1:
            positiveFlag = 1
            positiveIndex = cnt

        # 检查状态得在每轮结束时候，这样就能知道顺序了，减少一次比较
        # 一对点找到了

        if positiveFlag and negativeFlag:
            # print('状态',positiveFlag,negativeFlag,positiveIndex,negativeIndex)
            positiveFlag = 0
            negativeFlag = 0

            # print(math.fabs(positiveIndex-negativeIndex),FOV_threshold_pixel)
            if math.fabs(positiveIndex - negativeIndex) <= FOV_threshold_pixel:
                # 第二个是负，为正障碍物
                if i == -1:
                    print("正", positiveIndex, negativeIndex)
                    obstacle[positiveIndex:negativeIndex] = np.ones(negativeIndex - positiveIndex)
                # 第二个是正，为负障碍物
                if i == 1:
                    print("负", positiveIndex, negativeIndex)
                    obstacle[negativeIndex:positiveIndex] = -np.ones(positiveIndex - negativeIndex)




# velodyne = r'F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000007.npy'
# odometry = r'F:\Small_Obstacle_Dataset\train\file_2\odometry\0000000007.npy'
#
# f = np.load(velodyne,allow_pickle=True)

root_path = r'F:\Small_Obstacle_Dataset\train\file_2\velodyne'
folders = os.listdir(root_path)

# 相机内外参
# 内参
Intrinsics = [692.653256, 0.0, 629.321381, 0.0,0.0, 692.653256, 330.685425, 0.0,0.0, 0.0, 1.0, 0.0]
Intrinsics = np.array(Intrinsics)
Intrinsics = Intrinsics.reshape(3,4)
# 外参
LIDAR_Camera_Extrinsics=[0.9999597319083615, 0.008895027259147145, -0.0011887185606297354, 0.2640770375728607,
-0.008948839513686982, 0.9982905130079093, -0.05775785582726691, -0.04083506390452385,
0.0006729287596996524, 0.05776616768026164, 0.9983299139254618, 0.008564146235585213,
0.0, 0.0, 0.0, 1.0]
LIDAR_Camera_Extrinsics = np.array( LIDAR_Camera_Extrinsics)
LIDAR_Camera_Extrinsics = LIDAR_Camera_Extrinsics.reshape(4,4)
# axis翻转算子
# 防止像素走向不对，需要乘一个
LIDAR_Rotate = [0,-1,0,0,0,-1,1,0,0]
LIDAR_Rotate = np.array(LIDAR_Rotate).reshape(3,3)

imgs = []
for f in folders:
    #print(f)
    seg_path = os.path.join(root_path, f)
    print(seg_path)
    img = np.load(seg_path,allow_pickle=True)
    imgs.append(img)


F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000000.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000001.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000002.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000003.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000004.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000005.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000006.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000007.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000008.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000009.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000010.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000011.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000012.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000013.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000014.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000

F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000275.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000276.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000277.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000278.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000279.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000280.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000281.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000282.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000283.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000284.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000285.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000286.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000287.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000288.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000289.npy
F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000

In [ ]:
# 处理所有读取的点云
# for a in imgs:
a = imgs[42]
x = a[:, 0]
y = a[:, 1]
k = x / y
thetas = np.arctan(k)

# 点云投影到平面
pixel = np.dot(np.dot(Intrinsics, LIDAR_Camera_Extrinsics), a[:, 0:3])
pixel = pixel[0:2] / pixel[2]
pixel = pixel.astype(int)
tmp = np.column_stack((a[:, 0:3], np.ones(a.shape[0])))
a[:, 0:3] = np.dot(np.dot(Intrinsics, LIDAR_Camera_Extrinsics), tmp.T).T
# 归一化
# a[:, 2:3],不然不能广播
a[:, 0:2] = a[:, 0:2] / a[:, 2:3]
# pixel = pixel.astype(int)

# array拼接
# 5 角度（弧度制）
a = np.column_stack((a, thetas))

# 注意 这里的距离不一定是L2的 作者没说
# 6 距离
dis = np.linalg.norm(a[:, 0:2], axis=1, keepdims=True, ord=2)
a = np.column_stack((a, dis))

# 7 是否跳变点 0不是 1正 -1负 2异常
a = np.column_stack((a, np.zeros_like(a[:, 0])))

# 8 是否障碍物点 0不是 1正 -1负 2异常
a = np.column_stack((a, np.zeros_like(a[:, 0])))

# 分割成ring
LIDAR_RINGS = 16
assert LIDAR_RINGS > 1
rings = []
for i in range(LIDAR_RINGS):
    mask = a[:, 4] == i
    rings.append(a[mask])

# 按角度给每个id环中的点分别排序
for i in range(LIDAR_RINGS):
    rings[i] = rings[i][np.argsort(rings[i][:, 5])]

# 遍历环中的点
# 注意 边界点未处理
LIDAR_FOV_RESOLUSION = math.radians(0.2)
# LIDAR_FOV_RESOLUSION = 0.003490658503988659
theta = LIDAR_FOV_RESOLUSION
assert rings[1].shape[0] >= 3, 'insufficient points in ring'
for l in range(LIDAR_RINGS):
    for i in range(rings[l].shape[0] - 2):
        p1 = rings[l][i][6]
        p2 = rings[l][i + 1][6]
        p3 = rings[l][i + 2][6]
        rings[l][i + 2][7] = conpare_third_point(p3, predict_third_point(p1, p2, theta))

# 遍历环中的点
# 注意 正负没有指定，需要有时间重新判定，
assert rings[1].shape[0] >= 3, 'insufficient points in ring'
for l in range(LIDAR_RINGS):
    find_obstacle_by_breakpoint(breakpoints=rings[l][:, 7], obstacle=rings[l][:, 8])

# 转换到像素平面
for ring in rings:
    # 防止像素走向不对，需要乘一个axis rotate
    ring[:, 0:3] = LIDAR_Rotate.dot(ring[:, 0:3].T).T
    # 雷达目标在摄像头坐标系下的坐标
    # [4*4].T * [2*4].T
    tmp = np.dot(LIDAR_Camera_Extrinsics, np.column_stack((ring[:, 0:3], np.ones(ring.shape[0]))).T).T
    # 归一化
    thistmp = tmp[:, 0:2] / tmp[:, 2:3]
    # 归一化摄像头坐标系坐标
    thistmp = np.column_stack((thistmp, np.ones(thistmp.shape[0])))
    # 乘以内参矩阵，转换到像素平面
    ring[:, 0:3] = Intrinsics[:, 0:3].dot(thistmp.T).T
    # 检查满足要求的点
    # 检查有多少点满足要求
    h = ring[:, 0] <= 1280
    k = ring[:, 0] >= 0
    y = ring[:, 1] <= 720
    u = ring[:, 1] >= 0
    # 过滤
    filted = ring[:, 0:2][(h * k * y * u)]
    # 可视化
#     plt.figure()
#     plt.plot(filted[:, 0], filted[:, 1])
